# 从wikipedia的API接口查询上市公司的简称

In [1]:
import warnings
warnings.filterwarnings('ignore')

import wikipedia
import re
import os

from langconv import *
import pandas as pd
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)


### 获取上市公司全称，用于合并全称和简称搜索表时的去重key

In [2]:
def get_company_name_list():
    lines=open('company_cn.txt','r',encoding='utf-8').readlines()
    list_data=[]
    print(len(lines))

    for i in lines:
        i = i.strip('\n')
        list_data.append(i)

    return list_data

list_data=get_company_name_list()

3590


### 将所有繁体句子转换为简体句子

In [3]:
def fan_to_jian(sentence_list):
    new_sentence_list=[]
    for item in sentence_list:
        new_item=''
        for inde,ch in enumerate(item):
            if ch>'\u4e00' or '\u9fa5'>ch:
                #print(ch)
                ch=ch.encode(encoding="utf-8")
                ch = Converter('zh-hans').convert(ch.decode('utf-8'))
                ch = ch.encode('utf-8').decode('utf-8')
                #print(ch)
            new_item=new_item+ch
        #print(new_item)
        new_sentence_list.append(new_item)
    return new_sentence_list

### 查看含有简称字段的条数：

In [4]:
def check_keyword_number(new_sentence_list):
    brief_num=0
    for index,item in enumerate(new_sentence_list):
        if re.search('[\u7B80][\u79F0]',item):
            brief_num=brief_num+1
            
    return brief_num

### 提取简称方法

In [5]:
# [\u4e00-\u9fa5] 中文所有字
def get_breif_name(company_name_list,sentence_list):

    result_list=[]
    name_list=[]
    seg_result_list=[]

    for company_name,item in zip(company_name_list,sentence_list):
        try:
            
            item_list_result=[]
            pattern = re.compile('[a-zA-Z]*[简][称][为]?[作]?.*?[\u4e00-\u9fa5]+[a-zA-Z]*')
            result = re.findall(pattern, item)
            if result==[]:
                continue
            for i in result:
                item_list_result.append(re.sub('[:：]','',i)[2:])
            #print(item_list_result)
            result_list.append(item_list_result)
            name_list.append(company_name)
            
        except Exception as e:
            continue
    
    #分割 和 或 连接词
    for result in result_list:
        item_list=[]
        for item in result:
            item_list=item_list+'及'.join('和'.join(item.split('或')).split('和')).split('及')
        
        seg_result_list.append(item_list)
    
    #去掉部分引号
    final_result_list=[]
    for item in seg_result_list:
        list_item=[]
        for i in item:
            #print(re.findall('[a-zA-z]*[\u4e00-\u9fa5]+[a-zA-z]*',i))
            list_item=list_item+[i.replace("'",'').replace(',','').replace('“','').replace('为','').replace('「','').replace('；','').replace('深交所','')]
        final_result_list.append(list_item)

    result_dict={'name':name_list,'brief':final_result_list}
    #print(result_dict)
    df=pd.DataFrame(result_dict)
    return df

# ——————————按全称搜索得到csv表————————————

In [6]:
def save_to_csv(list_data):

    # %u7B80%u79F0 简称
    # [\u4e00-\u9fa5] 中文所有字
    name_list=[]
    result_list=[]
    wikipedia.set_lang('zh')

    for company_name in list_data:
        try:
            sentence = wikipedia.summary(company_name, sentences=1, auto_suggest=False)
            name_list.append(company_name)
            result_list.append(sentence)
            
        except Exception as e:
            continue
    result={'name':name_list,'result':result_list}
    df = pd.DataFrame(result)
    df.to_csv('csv_table.csv')
    return 0

In [7]:
if not os.path.exists('csv_table.csv'):
    save_to_csv(list_data)
else:
    df=pd.read_csv('csv_table.csv')

sentence_list=list(df['result'])
name_list=list(df['name'])

### 繁体转简体操作

In [8]:
new_sentence_list=fan_to_jian(sentence_list)

result_dict={'name':name_list,'sentence':new_sentence_list}
clean_df=pd.DataFrame(result_dict)
clean_df.to_csv('clean_data.csv')
clean_df.shape

(409, 2)

In [9]:
check_keyword_number(new_sentence_list)

268

In [10]:
company_name_list = clean_df.iloc[:,0]
sentence_list=clean_df.iloc[:,1]
df=get_breif_name(company_name_list,new_sentence_list)
df.shape

(268, 2)

In [11]:
add_df=pd.read_csv('result.csv')

# ————————按简称搜索得到csv表——————————

In [12]:
#按简称搜索得到csv表
def save_to_csv_brief(brief_name_list,list_data):
    name_list = []
    result_list = []
    wikipedia.set_lang('zh')

    for brief_name,company_name in zip(brief_name_list,list_data):
        try:
            sentence = wikipedia.summary(brief_name, sentences=1, auto_suggest=False)
            print(sentence)

            name_list.append(company_name)
            result_list.append(sentence)
        except Exception as e:
            print(e)
            continue
    result = {'name': name_list, 'result': result_list}
    df = pd.DataFrame(result)
    df.to_csv('brief_csv_table.csv')
    return 0


In [13]:
import os
import pandas as pd

if not os.path.exists('brief_csv_table.csv'):
    save_to_csv(list_data)
else:
    df=pd.read_csv('brief_csv_table.csv')

sentence_list=list(df['result'])
name_list=list(df['name'])

### 繁体转简体操作

In [14]:
new_sentence_list=fan_to_jian(sentence_list)

In [15]:
result_dict={'name':name_list,'sentence':new_sentence_list}
clean_df=pd.DataFrame(result_dict)
clean_df.to_csv('brief_clean_data.csv')

In [16]:
check_keyword_number(new_sentence_list)

330

In [17]:
company_name_list = clean_df.iloc[:,0]
sentence_list=clean_df.iloc[:,1]
df=get_breif_name(company_name_list,new_sentence_list)
df.shape

(330, 2)

In [18]:
df.to_csv('result2.csv')

表1：add_df 268    表2： df  330

### 合并两张表

In [19]:
#pd.concat([obj1, obj2])的效果与obj1.append(obj2)是相同的；
#当axis=1时，pd.concat([obj1, obj2], axis=1)的效果与pd.merge(obj1, obj2, left_index=True, right_index=True, how='outer')是相同的
final_df=pd.concat( [df,add_df], axis=0 )
final_df

,Unnamed: 0,brief,name
0,NaN,[平安银行],平安银行股份有限公司
1,NaN,"[万科, 万科集团]",万科企业股份有限公司
2,NaN,"[G*ST国农, 深安达]",深圳中国农大科技股份有限公司
3,NaN,"[GST星源, 深原野]",深圳世纪星源股份有限公司
4,NaN,[神州高铁],神州高铁技术股份有限公司
5,NaN,"[深宝安A, 深宝安]",中国宝安集团股份有限公司
6,NaN,[飞亚达A],飞亚达(集团)股份有限公司
7,NaN,[深能源],深圳能源集团股份有限公司
8,NaN,[一致药业],国药集团一致药业股份有限公司
9,NaN,[富奥股份],富奥汽车零部件股份有限公司


In [20]:
final_df.drop_duplicates('name')

,Unnamed: 0,brief,name
0,NaN,[平安银行],平安银行股份有限公司
1,NaN,"[万科, 万科集团]",万科企业股份有限公司
2,NaN,"[G*ST国农, 深安达]",深圳中国农大科技股份有限公司
3,NaN,"[GST星源, 深原野]",深圳世纪星源股份有限公司
4,NaN,[神州高铁],神州高铁技术股份有限公司
5,NaN,"[深宝安A, 深宝安]",中国宝安集团股份有限公司
6,NaN,[飞亚达A],飞亚达(集团)股份有限公司
7,NaN,[深能源],深圳能源集团股份有限公司
8,NaN,[一致药业],国药集团一致药业股份有限公司
9,NaN,[富奥股份],富奥汽车零部件股份有限公司


In [21]:
end_df=final_df[['name','brief']]
end_df=end_df.drop_duplicates('name')
end_df.reset_index(drop=True,inplace=True)
end_df

,name,brief
0,平安银行股份有限公司,[平安银行]
1,万科企业股份有限公司,"[万科, 万科集团]"
2,深圳中国农大科技股份有限公司,"[G*ST国农, 深安达]"
3,深圳世纪星源股份有限公司,"[GST星源, 深原野]"
4,神州高铁技术股份有限公司,[神州高铁]
5,中国宝安集团股份有限公司,"[深宝安A, 深宝安]"
6,飞亚达(集团)股份有限公司,[飞亚达A]
7,深圳能源集团股份有限公司,[深能源]
8,国药集团一致药业股份有限公司,[一致药业]
9,富奥汽车零部件股份有限公司,[富奥股份]


In [22]:
end_df.to_csv('final_abbr.csv')

按上市公司全称搜索wikipedia共409条数据，搜索3590家上市公司，搜索成功百分比：11.4%，
其中含有简称词条的共268条，已成功提取含有简称的268条数据，409条数据中，提取占比65.5%

按上市公司简称搜索wikipedia共589条数据，搜索3590家上市公司，搜索成功百分比：16.4%，
其中含有简称词条的共330条，已成功提取含有简称的330条数据，589条数据中，提取占比56%

合并搜索后的两张表，去除重复的205条数据，剩余393条数据，成功提取占所搜3590家公司的393/3590=10.9%


增加简称搜索之后，新增了125条数据，效果提升125/268=46.6%